# compile posterior into a value-added catalog

In [ ]:
import os
import h5py
import fitsio
import numpy as np
from tqdm import tqdm
import astropy.table as aTable

In [2]:
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask

In [3]:
os.sys.path.append(os.getcwd().replace('/nb', '/bin'))
import svda as SVDA

In [4]:
import healpy


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

In [5]:
# target and redrcok outputs joined (see https://desi.lbl.gov/trac/wiki/ClusteringWG/LSScat/SV3/current_version)
#ff = fitsio.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/datcomb_bright_tarspecwdup_Alltiles.fits')
ff = fitsio.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_ANY_full.dat.fits')

In [6]:
# select only BGS targets
is_bgs_bright = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_BRIGHT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

is_bgs_faint = (
    (ff['COADD_FIBERSTATUS'] == 0) & 
    ((ff['SV3_BGS_TARGET'] & sv3_bgs_mask['BGS_FAINT']) != 0) & 
    (ff['SPECTYPE'] == 'GALAXY') & 
    (ff['Z_HP'] > 0.) & (ff['Z_HP'] < 0.6) & 
    (ff['ZWARN'] == 0) & 
    (ff['DELTACHI2'] > 40.) & 
    (ff['ZERR'] < 0.0005 * (1 + ff['Z_HP'])))

print('%i BGS BRIGHT z-success' % np.sum(is_bgs_bright))
print('%i BGS FAINT z-success' % np.sum(is_bgs_faint))

143074 BGS BRIGHT z-success
96771 BGS FAINT z-success


In [7]:
vagc = aTable.Table()

columns = ['TARGETID', 'RA', 'DEC', 'Z_HP', 'ZERR', 'TSNR2_BGS']
for col in columns: 
    vagc[col] = ff[col]

In [8]:
for col in ['MAG_G', 'MAG_R', 'MAG_Z', 'MAG_W1', 'FIBMAG_R']: 
    vagc[col] = np.repeat(-999., len(vagc))
vagc['HPIX_64'] = np.repeat(-999, len(vagc)).astype(int)

In [9]:
for subsample in [is_bgs_bright, is_bgs_faint]: 

    photsys = ff['PHOTSYS'][subsample].astype(str)
    trans_g  = SVDA.mwdust_transmission(ff['EBV'][subsample], 'g', photsys, match_legacy_surveys=False)
    trans_r  = SVDA.mwdust_transmission(ff['EBV'][subsample], 'r', photsys, match_legacy_surveys=False)
    trans_z  = SVDA.mwdust_transmission(ff['EBV'][subsample], 'z', photsys, match_legacy_surveys=False)
    trans_w1 = SVDA.mwdust_transmission(ff['EBV'][subsample], 'z', photsys, match_legacy_surveys=False)

    flux_g  = ff['FLUX_G'][subsample] / trans_g
    flux_r  = ff['FLUX_R'][subsample] / trans_r
    flux_z  = ff['FLUX_Z'][subsample] / trans_z
    flux_w1 = ff['FLUX_W1'][subsample] / trans_w1
    fiberflux_r = ff['FIBERFLUX_R'][subsample] / trans_r

    vagc['MAG_G'][subsample]  = 22.5 - 2.5 * np.log10(flux_g.clip(1e-7))
    vagc['MAG_R'][subsample]  = 22.5 - 2.5 * np.log10(flux_r.clip(1e-7))
    vagc['MAG_Z'][subsample]  = 22.5 - 2.5 * np.log10(flux_z.clip(1e-7))
    vagc['MAG_W1'][subsample] = 22.5 - 2.5 * np.log10(flux_w1.clip(1e-7))

    vagc['FIBMAG_R'][subsample] = 22.5 - 2.5 * np.log10(fiberflux_r.clip(1e-7))
    
    phi = np.radians(vagc['RA'])[subsample]
    theta = np.radians(90. - vagc['DEC'])[subsample]
    vagc['HPIX_64'][subsample] = healpy.ang2pix(64, theta, phi, nest=True)

In [10]:
uhpix = list(np.unique(vagc['HPIX_64']))
uhpix.remove(-999)

In [11]:
vagc['provabgs_mcmc'] = np.tile(-999., (len(vagc), 100, 13))
vagc['provabgs_theta_bf'] = np.tile(-999., (len(vagc), 13))
vagc['provabgs_z_max'] = np.tile(-999., len(vagc))
vagc['provabgs_logMstar_bf'] = np.tile(-999., len(vagc))
vagc['provabgs_logMstar'] = np.tile(-999., (len(vagc), 100))

In [13]:
dat_dir = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/'
for hpix in tqdm(uhpix): 
    for targ in ['BRIGHT', 'FAINT']: 
        fhpix = os.path.join(dat_dir, f'provabgs-sv3-bright-{hpix}.BGS_{targ}.hdf5')    
        if not os.path.isfile(fhpix): continue 
        #try: 
        with h5py.File(fhpix, 'r') as fhpix: 
            tids = fhpix['targetid'][...] # target id
            mcmc  = fhpix['samples'][...] # posterior samples
            mcmc  = mcmc.reshape((mcmc.shape[0], mcmc.shape[1] * mcmc.shape[2], mcmc.shape[3]))
            ttbf = fhpix['theta_bf'][...]

            zmax = fhpix['zmax'][...]
            logMs_bf = fhpix['logMstar_bf'][...]
            logMstar = fhpix['logMstar'][...]

        ngals = len(tids)
        for igal in range(ngals): 
            is_target = (vagc['TARGETID'] == tids[igal])
            if np.sum(is_target) > 1: 
                raise ValueError #print(np.sum(is_target), hpix, tids[igal])

            vagc['provabgs_mcmc'][is_target]     = mcmc[igal,-100:,:]
            vagc['provabgs_theta_bf'][is_target] = ttbf[igal]
            vagc['provabgs_z_max'][is_target]    = zmax[igal]

            vagc['provabgs_logMstar_bf'][is_target] = logMs_bf[igal]
            vagc['provabgs_logMstar'][is_target]    = logMstar[igal,-100:]

100%|██████████| 94/94 [00:00<00:00, 623.59it/s]


In [20]:
fout = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_ANY_full.provabgs.hdf5'
vagc.write(fout, overwrite=True)

/global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  "using default path {}".format(path))


## compile BGS Bright

In [3]:
dir_dat = '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/'
_bgs_n = aTable.Table.read(os.path.join(dir_dat, 'BGS_BRIGHT_N_clustering.dat.fits'))
_bgs_s = aTable.Table.read(os.path.join(dir_dat, 'BGS_BRIGHT_S_clustering.dat.fits'))

bgs_bright = aTable.vstack([_bgs_n, _bgs_s])

In [4]:
len(bgs_bright)

143853

In [5]:
bgs_bright[:5]

RA,DEC,TARGETID,NTILE,TILES,Z,COMP_TILE,rosette_number,rosette_r,FRACZ_TILELOCID,BITWEIGHTS [2],PROB_OBS,WEIGHT_ZFAIL,WEIGHT,flux_g_dered,flux_r_dered,flux_z_dered,flux_w1_dered,flux_w2_dered,NZ,WEIGHT_FKP,REST_GMR_0P1,KCORR_R0P1,KCORR_G0P1,KCORR_R0P0,KCORR_G0P0,REST_GMR_0P0,EQ_ALL_0P0,EQ_ALL_0P1,ABSMAG_R
float64,float64,int64,int64,bytes43,float64,float64,float64,float64,float64,int64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
191.20458196461243,60.31535787110577,39633390185481345,1,100,0.08969256109731599,0.64,3.0,1.613220114177946,1.0,-1 .. -1,1.0,1.0,1.0,11.618794,18.39226,20.892622,13.421043,9.27371,0.04778151135868713,0.0030821516772042766,0.5107507237961553,-0.10918667636401896,-0.12125347071681446,0.09189331428141073,0.11205224487489494,0.47852499884987515,-0.08700178426439652,-0.08700178426439652,-17.752532357564764
187.04622815471808,60.82670180284912,39633396124617897,1,100,0.37400455377945924,0.64,3.0,1.6801915534232357,1.0,-1 .. -1,1.0,1.0,1.0,4.046807,16.82531,38.79777,85.02852,54.11833,0.0009577827729000268,0.133626516949366,0.8657535543881233,0.2757199965785285,0.9570932122099367,0.5034037640174682,1.319054690619937,0.7314758434170625,-0.36278441716607546,-0.36278441716607546,-21.20717884549835
187.83760708675698,60.38987543677365,39633393155049303,1,100,0.1731108191825362,0.64,3.0,1.7267282368438097,0.5,6932875859169517021 .. -6858338461939850770,0.4609375,1.0,2.15,13.26433,27.728409,46.020153,37.03337,25.867756,0.01270745015908088,0.01149149450827468,0.6595463164766608,-0.04866411688213576,0.09238961852108621,0.17196093424278175,0.3785062363399163,0.5940547497827482,-0.16791749460706012,-0.16791749460706012,-19.719994580683654
187.05117249536465,62.614542270156655,39633416324383107,1,100,0.2703579310142208,0.64,3.0,1.5579854421783794,1.0,-1 .. -1,1.0,1.0,1.0,4.4575434,17.798513,37.420948,46.075268,30.963612,0.004051015958871262,0.035183206585599834,0.9961770755006489,0.146409873660811,0.6534524026523496,0.3797334250181883,1.0502045184025013,0.8327485111078745,-0.2622471930837942,-0.2622471930837942,-20.42505494547267
191.14757099005186,60.345815959235,39633390185480938,1,100,0.2819459922463374,0.64,3.0,1.5743490921284766,0.5,4864741233190265570 .. 434972757299228716,0.359375,1.0,2.74468085106383,6.782468,16.732075,29.996437,31.939342,25.343126,0.004051015958871262,0.035183206585599834,0.6031055527358372,-0.0007029256786438565,0.3765973290806362,0.21603985022885944,0.6418090846178732,0.5546365731061034,-0.27348761247894726,-0.27348761247894726,-20.30376568408361


In [6]:
bgs_bright['phi'] = np.radians(bgs_bright['RA'])
bgs_bright['theta'] = np.radians(90. - bgs_bright['DEC'])
bgs_bright['healpix'] = healpy.ang2pix(64, bgs_bright['theta'], bgs_bright['phi'], nest=True)

get all healpixs

In [7]:
tiles_fuji = aTable.Table.read('/global/cfs/cdirs/desi/spectro/redux/fuji/healpix/tilepix.fits')
is_bright = (tiles_fuji['PROGRAM'] == 'bright')
is_sv3 = (tiles_fuji['SURVEY'] == 'sv3')

hpixs = np.unique(np.sort(np.array(tiles_fuji['HEALPIX'][is_bright & is_sv3])))

In [8]:
mstars = np.tile(-999., (len(bgs_bright), 30))
zmaxes = np.repeat(-999., len(bgs_bright))
theta_bfs = np.tile(-999., (len(bgs_bright), 13))

for hpix in hpixs: 
    try: 
        f = h5py.File('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/provabgs-sv3-bright-%s.BGS_BRIGHT.hdf5' % hpix, 'r')
        tid = f['targetid'][...]
        mcmc = f['samples'][...][:,-1,:,0]
        zmax = f['zmax'][...]
        ttbf = f['theta_bf'][...]
    except IndexError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except KeyError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except OSError: 
        print('no posteriors for %i' % hpix)
        continue
    except: 
        print('problem reading %i' % hpix)
        continue 
    
    is_hpix = (bgs_bright['healpix'] == hpix)

    for i in np.arange(len(bgs_bright))[is_hpix]: 
        is_target = (tid == bgs_bright['TARGETID'][i])

        if np.sum(is_target) == 1:
            mstars[i,:] = mcmc[is_target]
            zmaxes[i] = zmax[is_target]
            theta_bfs[i,:] = ttbf[is_target] 

no BGS BRIGHT galaxies in 9932
no posteriors for 9985
no posteriors for 10520
no posteriors for 25939
no posteriors for 26087


In [9]:
bgs_bright['logM_mcmc'] = mstars
bgs_bright['zmax'] = zmaxes
bgs_bright['theta_bf'] = theta_bfs

In [10]:
print("%i galaxies don't have posteriors" % np.sum(mstars[:,0] == -999.))

1341 galaxies don't have posteriors


In [11]:
bgs_bright[:5]

RA,DEC,TARGETID,NTILE,TILES,Z,COMP_TILE,rosette_number,rosette_r,FRACZ_TILELOCID,BITWEIGHTS [2],PROB_OBS,WEIGHT_ZFAIL,WEIGHT,flux_g_dered,flux_r_dered,flux_z_dered,flux_w1_dered,flux_w2_dered,NZ,WEIGHT_FKP,REST_GMR_0P1,KCORR_R0P1,KCORR_G0P1,KCORR_R0P0,KCORR_G0P0,REST_GMR_0P0,EQ_ALL_0P0,EQ_ALL_0P1,ABSMAG_R,phi,theta,healpix,logM_mcmc [30],zmax,theta_bf [13]
float64,float64,int64,int64,bytes43,float64,float64,float64,float64,float64,int64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64
191.20458196461243,60.31535787110577,39633390185481345,1,100,0.08969256109731599,0.64,3.0,1.613220114177946,1.0,-1 .. -1,1.0,1.0,1.0,11.618794,18.39226,20.892622,13.421043,9.27371,0.04778151135868713,0.0030821516772042766,0.5107507237961553,-0.10918667636401896,-0.12125347071681446,0.09189331428141073,0.11205224487489494,0.47852499884987515,-0.08700178426439652,-0.08700178426439652,-17.752532357564764,3.3371495001818547,0.5180947424254233,11226,9.311141344469839 .. 9.30371384223851,0.244537973921384,9.299830413616577 .. 0.12427289814965596
187.04622815471808,60.82670180284912,39633396124617897,1,100,0.37400455377945924,0.64,3.0,1.6801915534232357,1.0,-1 .. -1,1.0,1.0,1.0,4.046807,16.82531,38.79777,85.02852,54.11833,0.0009577827729000268,0.133626516949366,0.8657535543881233,0.2757199965785285,0.9570932122099367,0.5034037640174682,1.319054690619937,0.7314758434170625,-0.36278441716607546,-0.36278441716607546,-21.20717884549835,3.264572534736348,0.5091701072064087,11237,11.532893592539585 .. 11.652536919740712,0.5180062804791324,11.529095176742617 .. 0.3710937098062413
187.83760708675698,60.38987543677365,39633393155049303,1,100,0.1731108191825362,0.64,3.0,1.7267282368438097,0.5,6932875859169517021 .. -6858338461939850770,0.4609375,1.0,2.15,13.26433,27.728409,46.020153,37.03337,25.867756,0.01270745015908088,0.01149149450827468,0.6595463164766608,-0.04866411688213576,0.09238961852108621,0.17196093424278175,0.3785062363399163,0.5940547497827482,-0.16791749460706012,-0.16791749460706012,-19.719994580683654,3.2783847027313433,0.5167941655539476,11237,10.562881907340254 .. 10.51416114213185,0.3653015609857238,10.502975152735333 .. 0.2744993206027024
187.05117249536465,62.614542270156655,39633416324383107,1,100,0.2703579310142208,0.64,3.0,1.5579854421783794,1.0,-1 .. -1,1.0,1.0,1.0,4.4575434,17.798513,37.420948,46.075268,30.963612,0.004051015958871262,0.035183206585599834,0.9961770755006489,0.146409873660811,0.6534524026523496,0.3797334250181883,1.0502045184025013,0.8327485111078745,-0.2622471930837942,-0.2622471930837942,-20.42505494547267,3.264658829759971,0.47796640455149814,11251,11.164636307948935 .. 11.122322575621661,0.3243951220160425,11.16028636319183 .. 0.6710591095679554
191.14757099005186,60.345815959235,39633390185480938,1,100,0.2819459922463374,0.64,3.0,1.5743490921284766,0.5,4864741233190265570 .. 434972757299228716,0.359375,1.0,2.74468085106383,6.782468,16.732075,29.996437,31.939342,25.343126,0.004051015958871262,0.035183206585599834,0.6031055527358372,-0.0007029256786438565,0.3765973290806362,0.21603985022885944,0.6418090846178732,0.5546365731061034,-0.27348761247894726,-0.27348761247894726,-20.30376568408361,3.3361544709660023,0.5175631485037057,11226,11.088944348844176 .. 11.20103049582311,0.42353546824905075,11.14249424405227 .. 0.3651476796540595


In [12]:
bgs_bright.write('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_BRIGHT_clustering.sv3.logM.hdf5', overwrite=True)

/global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  "using default path {}".format(path))


# compile BGS Faint

In [16]:
dir_dat = '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/'
_bgs_n = aTable.Table.read(os.path.join(dir_dat, 'BGS_ANY_N_clustering.dat.fits'))
_bgs_s = aTable.Table.read(os.path.join(dir_dat, 'BGS_ANY_S_clustering.dat.fits'))

bgs_any = aTable.vstack([_bgs_n, _bgs_s])
print('%i BGS All galaxies' % len(bgs_any))

241746 BGS All galaxies


In [17]:
bgs_any['phi'] = np.radians(bgs_any['RA'])
bgs_any['theta'] = np.radians(90. - bgs_any['DEC'])
bgs_any['healpix'] = healpy.ang2pix(64, bgs_any['theta'], bgs_any['phi'], nest=True)

In [18]:
mstars = np.tile(-999., (len(bgs_any), 30))
zmaxes = np.repeat(-999., len(bgs_any))
theta_bfs = np.tile(-999., (len(bgs_any), 13))

for hpix in hpixs: 
    try: 
        f = h5py.File('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/provabgs-sv3-bright-%s.BGS_BRIGHT.hdf5' % hpix, 'r')
        tid = f['targetid'][...]
        mcmc = f['samples'][...][:,-1,:,0]
        zmax = f['zmax'][...]
        ttbf = f['theta_bf'][...]
    except IndexError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except KeyError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except OSError: 
        print('no posteriors for %i' % hpix)
        continue
    except: 
        print('problem reading %i' % hpix)
        continue 
    
    is_hpix = (bgs_any['healpix'] == hpix)

    for i in np.arange(len(bgs_any))[is_hpix]: 
        is_target = (tid == bgs_any['TARGETID'][i])

        if np.sum(is_target) == 1:
            mstars[i,:] = mcmc[is_target]
            zmaxes[i] = zmax[is_target]
            theta_bfs[i,:] = ttbf[is_target] 
    
    try: 
        f = h5py.File('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/provabgs-sv3-bright-%s.BGS_FAINT.hdf5' % hpix, 'r')
        tid = f['targetid'][...]
        mcmc = f['samples'][...][:,-1,:,0]
        zmax = f['zmax'][...]
        ttbf = f['theta_bf'][...]
    except IndexError: 
        print('no BGS FAINT galaxies in %i' % hpix)
        continue
    except KeyError: 
        print('no BGS FAINT galaxies in %i' % hpix)
        continue
    except OSError: 
        print('no posteriors for %i' % hpix)
        continue
    except: 
        print('problem reading %i' % hpix)
        continue 
    
    for i in np.arange(len(bgs_any))[is_hpix]: 
        is_target = (tid == bgs_any['TARGETID'][i])

        if np.sum(is_target) == 1:
            mstars[i,:] = mcmc[is_target]
            zmaxes[i] = zmax[is_target]
            theta_bfs[i,:] = ttbf[is_target] 
            
bgs_any['logM_mcmc'] = mstars
bgs_any['zmax'] = zmaxes
bgs_any['theta_bf'] = theta_bfs

no posteriors for 9791
no BGS BRIGHT galaxies in 9932
no posteriors for 9983
no posteriors for 9985
no posteriors for 10198
no posteriors for 10520
no posteriors for 11936
no posteriors for 25939
no posteriors for 26087
no posteriors for 27238
no posteriors for 27257
no posteriors for 28115
no posteriors for 28126


In [19]:
print("%i BGS ANY galaxies don't have posteriors" % np.sum(mstars[:,0] == -999.))

3786 BGS ANY galaxies don't have posteriors


In [20]:
bgs_any[:5]

RA,DEC,TARGETID,NTILE,TILES,Z,COMP_TILE,rosette_number,rosette_r,FRACZ_TILELOCID,BITWEIGHTS [2],PROB_OBS,WEIGHT_ZFAIL,WEIGHT,flux_g_dered,flux_r_dered,flux_z_dered,flux_w1_dered,flux_w2_dered,NZ,WEIGHT_FKP,REST_GMR_0P1,KCORR_R0P1,KCORR_G0P1,KCORR_R0P0,KCORR_G0P0,REST_GMR_0P0,EQ_ALL_0P0,EQ_ALL_0P1,ABSMAG_R,phi,theta,healpix,logM_mcmc [30],zmax,theta_bf [13]
float64,float64,int64,int64,bytes43,float64,float64,float64,float64,float64,int64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64
191.14757099005186,60.345815959235,39633390185480938,1,100,0.2819459922463374,0.4318181818181818,3.0,1.5743490921284766,0.3333333333333333,4864741233190265570 .. 434972757299228716,0.359375,1.0,2.74468085106383,6.782468,16.732075,29.996437,31.939342,25.343126,0.007685987687919836,0.018919347320389374,0.6031055527358372,-0.0007029256786438565,0.3765973290806362,0.21603985022885944,0.6418090846178732,0.5546365731061034,-0.27348761247894726,-0.27348761247894726,-20.30376568408361,3.3361544709660023,0.5175631485037057,11226,11.088944348844176 .. 11.20103049582311,0.42353546824905075,11.14249424405227 .. 0.3651476796540595
187.05117249536465,62.614542270156655,39633416324383107,1,100,0.2703579310142208,0.4318181818181818,3.0,1.5579854421783794,1.0,-1 .. -1,1.0,1.0,1.0,4.4575434,17.798513,37.420948,46.075268,30.963612,0.007685987687919836,0.018919347320389374,0.9961770755006489,0.146409873660811,0.6534524026523496,0.3797334250181883,1.0502045184025013,0.8327485111078745,-0.2622471930837942,-0.2622471930837942,-20.42505494547267,3.264658829759971,0.47796640455149814,11251,11.164636307948935 .. 11.122322575621661,0.3243951220160425,11.16028636319183 .. 0.6710591095679554
187.83760708675698,60.38987543677365,39633393155049303,1,100,0.1731108191825362,0.4318181818181818,3.0,1.7267282368438097,0.2,6932875859169517021 .. -6858338461939850770,0.4609375,1.0,2.15,13.26433,27.728409,46.020153,37.03337,25.867756,0.017899230975836122,0.00821268752302798,0.6595463164766608,-0.04866411688213576,0.09238961852108621,0.17196093424278175,0.3785062363399163,0.5940547497827482,-0.16791749460706012,-0.16791749460706012,-19.719994580683654,3.2783847027313433,0.5167941655539476,11237,10.562881907340254 .. 10.51416114213185,0.3653015609857238,10.502975152735333 .. 0.2744993206027024
188.7298325265811,60.181046704454054,39633390164510257,1,100,0.2854181586411897,0.4318181818181818,3.0,1.7154596184117625,0.3333333333333333,-1116572791947434369 .. -1544975305178865643,0.4296875,1.0,2.3035714285714284,9.799708,23.07847,40.24855,43.608025,32.920425,0.007685987687919836,0.018919347320389374,0.5506603859408487,-0.027984211522950803,0.35134044892291943,0.17986420125298275,0.5984338493516907,0.5114153982880107,-0.276855613881954,-0.276855613881954,-20.65269989753773,3.293956974326329,0.5204389145056906,11237,11.36773484584262 .. 11.267410531148103,0.46391551779465107,11.257075728183183 .. 0.3983021711356034
187.04622815471808,60.82670180284912,39633396124617897,1,100,0.37400455377945924,0.4318181818181818,3.0,1.6801915534232357,1.0,-1 .. -1,1.0,1.0,1.0,4.046807,16.82531,38.79777,85.02852,54.11833,0.0025147845197826895,0.05565824857525882,0.8657535543881233,0.2757199965785285,0.9570932122099367,0.5034037640174682,1.319054690619937,0.7314758434170625,-0.36278441716607546,-0.36278441716607546,-21.20717884549835,3.264572534736348,0.5091701072064087,11237,11.532893592539585 .. 11.652536919740712,0.5180062804791324,11.529095176742617 .. 0.3710937098062413


In [22]:
bgs_any.write('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_ANY_clustering.sv3.logM.hdf5', overwrite=True)